In [2]:
import dspy
import os
from dotenv import load_dotenv
load_dotenv()
load_dotenv('.secrets')

/Users/vicente/Library/Caches/pypoetry/virtualenvs/gen-ai-demos-umKuaqgj-py3.12/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
/Users/vicente/Library/Caches/pypoetry/virtualenvs/gen-ai-demos-umKuaqgj-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
class BoardGameResearchPlanner(dspy.Signature):
    question = dspy.InputField(desc="The user's initial query about board games")
    subquestions: list[str] = dspy.OutputField(desc="A list of subquestions that get deeper into the theme the user is making research")

class BoardGameResearcher(dspy.Signature):
    question = dspy.InputField(desc="The user's initial query about board games")
    subquestion: str = dspy.InputField(desc="The subquestion you are doing deep research on")
    context = dspy.OutputField(desc="The context learned by answering the subquestion while relevant to the initial question")

class BoardGameQuestionAnswer(dspy.Signature):
    question = dspy.InputField(desc="The user's initial query about board games")
    answer = dspy.OutputField(desc="The final answer including all the context researched about board games.")

class BoardGameResearchSummarizer(dspy.Signature):
    """Condense retrieved information into a focused summary."""
    question = dspy.InputField(desc="The user's initial query about board games")
    context: list[str] = dspy.InputField(desc="Research context discovered about the board games you are about to recommend")
    summary = dspy.OutputField(desc="Condense retrieved information into a focused summary relevant to the provided context and user question.")


class BoardGameAssistant(dspy.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.plan_research = dspy.ChainOfThought(BoardGameResearchPlanner)
        self.research = dspy.ChainOfThought(BoardGameResearcher)
        self.summarize = dspy.ChainOfThought(BoardGameResearchSummarizer)
        self.answer = dspy.ChainOfThought(BoardGameQuestionAnswer)

    def forward(self, question):
        subquestions = self.plan_research(question=question).subquestions
        research_context = []
        print(f"Question: {question} research tasks: {len(subquestions)}")
        for subquestion in subquestions:
            print(f" - Researching: {subquestion}")
            research = self.research(subquestion=subquestion, question=question)
            research_context = research_context + [research.context]

        final_research = self.summarize(context=research_context, question=question)

        return dspy.Prediction(answer=final_research.summary)

In [4]:
lm = dspy.LM('openai/gpt-4o', api_key=os.getenv("OPENAI_API_KEY"))
dspy.configure(lm=lm)

In [5]:
assistant = BoardGameAssistant()
recommendation = assistant(question="What is the best opening move in Catan?")

Question: What is the best opening move in Catan? research tasks: 6
 - Researching: How does the distribution of numbers on the board affect the best opening move in Catan?
 - Researching: What are the most important resources to secure in the opening move of Catan?
 - Researching: How does the number of players influence the best opening strategy in Catan?
 - Researching: What are common mistakes to avoid when making the opening move in Catan?
 - Researching: How can the initial placement of settlements impact future trading opportunities in Catan?
 - Researching: What strategies do experienced players use for their opening moves in Catan?


In [15]:
recommendation


Prediction(
    answer='The best opening move in Catan involves placing initial settlements on intersections with high-frequency numbers like 6 and 8 to ensure consistent resource production. Players should prioritize access to wood, brick, wheat, and sheep for early expansion. The strategy should adapt to the number of players, with more selectivity in a 3-player game and a balanced approach in a 4-player game. Avoiding common mistakes, such as neglecting resource balance and future growth, is crucial. Strategic placement should also consider long-term trading potential and expansion opportunities.'
)

In [6]:
lm.inspect_history(n=10)





[2025-02-02T12:56:13.734116]

System message:

Your input fields are:
1. `question` (str): The user's initial query about board games

Your output fields are:
1. `reasoning` (str)
2. `subquestions` (list[str]): A list of subquestions that get deeper into the theme the user is making research

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## subquestions ## ]]
{subquestions}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `subquestions`.


User message:

[[ ## question ## ]]
What is the best opening move in Catan?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## subquestions ## ]]` 